## Consensus Clusters on Louvain Algorithm

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from neuprint import  fetch_adjacencies, fetch_neurons
from neuprint.utils import connection_table_to_matrix, merge_neuron_properties

In [ ]:
import bct
from netneurotools import cluster
from netneurotools import modularity

In [4]:
# DNp01 (giant fiber) to DNp11
body_ids = ["2307027729","5813024015", "1565846637", "1405231475", "1466998977", "5813023322", "1100404581", "1226887763", "1228264951", "512851433", "5813026936", "1281324958"]
DNp_ids = [int(i) for i in body_ids]

In [21]:
# read all_connection_df.csv file
all_connection_df = pd.read_csv("./data/all_connection_df.csv")

In [22]:
def make_matrix(conn_df, group_cols='bodyId', weight_col='weight', sort_by=None, make_square=False):
    if isinstance(group_cols, str):
        group_cols = (f"{group_cols}_pre", f"{group_cols}_post")

    assert len(group_cols) == 2, \
        "Please provide two group_cols (e.g. 'bodyId_pre', 'bodyId_post')"

    assert group_cols[0] in conn_df, \
        f"Column missing: {group_cols[0]}"

    assert group_cols[1] in conn_df, \
        f"Column missing: {group_cols[1]}"

    assert weight_col in conn_df, \
        f"Column missing: {weight_col}"

    col_pre, col_post = group_cols
    dtype = conn_df[weight_col].dtype

    agg_weights_df = conn_df.groupby([col_pre, col_post], sort=False)[weight_col].sum().reset_index()
    matrix = agg_weights_df.pivot(col_pre, col_post, weight_col)
    matrix = matrix.fillna(0).astype(dtype)

    if sort_by:
        if isinstance(sort_by, str):
            sort_by = (f"{sort_by}_pre", f"{sort_by}_post")

        assert len(sort_by) == 2, \
            "Please provide two sort_by column names (e.g. 'type_pre', 'type_post')"

        pre_order = conn_df.sort_values(sort_by[0])[col_pre].unique()
        post_order = conn_df.sort_values(sort_by[1])[col_post].unique()
        matrix = matrix.reindex(index=pre_order, columns=post_order)
    else:
        # No sort: Keep the order as close to the input order as possible.
        pre_order = conn_df[col_pre].unique()
        post_order = conn_df[col_post].unique()
        matrix = matrix.reindex(index=pre_order, columns=post_order)

    if make_square:    
        matrix ,_ = matrix.align(matrix.T)
        matrix = matrix.fillna(0.0).astype(matrix.dtypes) # not sure abt dtypes

        # matrix, _ = matrix.align(matrix.T).fillna(0.0).astype(matrix.dtype)
        matrix = matrix.rename_axis('bodyId_pre', axis=0).rename_axis('bodyId_post', axis=1)
        matrix = matrix.loc[sorted(matrix.index), sorted(matrix.columns)]

    return matrix

In [23]:
# convert to NxN adjacency matrix
adj_mat_df = make_matrix(all_connection_df, 'bodyId', make_square=True)

/var/folders/q5/wpyng01x7nld79qn65091y_r0000gn/T/ipykernel_3901/2387842939.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  matrix = agg_weights_df.pivot(col_pre, col_post, weight_col)


In [25]:
adj_mat = adj_mat_df.to_numpy()

In [163]:
""" ci = []
for i in range(10):
    # run bct's Louvain algorithm
    ci_assign, q = bct.community_louvain(adj_mat)
    ci.append(ci_assign)
    
    # get the number of communities
    n_communities = len(np.unique(ci_assign))
    print('{} clusters detected with a modularity of {:.2f}.'.format(n_communities, q)) """

BCTParamError: Modularity infinite loop style G. Please contact the developer.

In [173]:
# ci = [bct.community_louvain(adj_mat)[0] for n in range(5)]

In [26]:
from sknetwork.clustering import Louvain, get_modularity
from sknetwork.linalg import normalize
from sknetwork.utils import get_membership

from sknetwork.data import from_edge_list
from scipy import sparse

import networkx as nx

In [27]:
dfFilt = all_connection_df[['bodyId_pre', 'bodyId_post', 'weight']] # sknetwork uses 3rd col as weight

In [28]:
graph = from_edge_list(list(dfFilt.itertuples(index=False)), weighted=True, directed=True) # without directed=True, wrong # of elements

In [29]:
graph.adjacency, graph.names

(<5596x5596 sparse matrix of type '<class 'numpy.int64'>'
 	with 389065 stored elements in Compressed Sparse Row format>,
 array([ 326253554,  357245785,  357249472, ..., 7112615304, 7112622763,
        7112624834]))

In [30]:
# find DNp ids in consensus clusters
bodyId_indices = [np.where(graph.names == neuron)[0][0] for neuron in DNp_ids] # indices of bodyIds in graph.names 

NOTE: an implicit assumption in Louvain's clustering assignment output is that clustering labels are assigned to nodes in the order present in graph.names.

In [31]:
consensusResults = [] # this list stores a tuple for each iteration: (numIter, consensusClusterAssn)

In [72]:
expIters = 250 # should be multiples of 10 (maybe 5 is ok too)
clusterAssignments = []

for _ in range(expIters):
    louvain = Louvain(shuffle_nodes=True)
    labels = louvain.fit_predict(graph.adjacency) # each node (neuron) is assigned a label
    clusterAssignments.append(labels)

    labels_unique, counts = np.unique(labels, return_counts=True)
    if expIters > 10 and expIters % 10 == 0:
        continue
    print(labels_unique, counts)

[0 1 2 3 4 5 6] [1372 1112 1038  927  786  291   70]
[0 1 2 3 4 5 6] [1273 1079 1057 1023  651  306  207]
[0 1 2 3 4 5 6] [1672 1352 1309  750  287  155   71]
[0 1 2 3 4 5 6 7] [1176 1096 1066 1022  769  392   73    2]
[0 1 2 3 4 5 6] [1524 1007  973  853  584  388  267]
[0 1 2 3 4 5 6] [1091 1080 1078 1055  946  276   70]
[0 1 2 3 4 5 6 7] [1279 1057 1049  894  821  279  146   71]
[0 1 2 3 4 5 6] [1732 1599 1328  388  315  164   70]
[0 1 2 3 4 5 6] [1307 1170 1001  998  782  268   70]
[0 1 2 3 4 5 6] [1673 1354 1352  711  280  156   70]
[0 1 2 3 4 5 6] [1668 1285 1084 1060  277  152   70]
[0 1 2 3 4 5] [1671 1261 1079 1066  316  203]
[0 1 2 3 4 5 6] [1073 1073 1068 1048  988  276   70]
[0 1 2 3 4 5 6] [1231 1097  970  898  733  396  271]
[0 1 2 3 4 5 6 7] [1610 1058 1053 1030  419  269  155    2]
[0 1 2 3 4 5 6] [1112 1109 1076 1005  938  285   71]
[0 1 2 3 4 5] [1687 1423 1076 1045  295   70]
[0 1 2 3 4 5] [1628 1346 1065  898  389  270]
[0 1 2 3 4 5] [1690 1264 1075  902  395  270]


In [73]:
clusterAssignments = np.array(clusterAssignments)
clusterAssignments

array([[4, 4, 4, ..., 1, 1, 5],
       [4, 4, 4, ..., 1, 1, 5],
       [0, 0, 0, ..., 1, 1, 4],
       ...,
       [4, 4, 4, ..., 1, 1, 5],
       [1, 1, 1, ..., 0, 0, 5],
       [0, 0, 0, ..., 2, 2, 6]])

In [76]:
# consensus = cluster.find_consensus(np.column_stack(clusterAssignments), seed=1234)
consensus = cluster.find_consensus(np.column_stack(clusterAssignments))

In [77]:
# find unique labels and their counts
labels_unique, counts = np.unique(consensus, return_counts=True)
print(labels_unique, counts)

[1 2 3 4 5] [1705 1286 1260  268 1077]


In [79]:
for i in bodyId_indices:
    print(consensus[i], end=', ')

5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5, 

In [80]:
consensusResults.append((expIters, consensus))

In [85]:
consensusResults

[(10, array([1, 1, 1, ..., 4, 4, 3])),
 (20, array([1, 1, 1, ..., 4, 4, 3])),
 (50, array([1, 1, 1, ..., 5, 5, 4])),
 (100, array([1, 1, 1, ..., 5, 5, 4])),
 (250, array([1, 1, 1, ..., 5, 5, 4]))]

In [92]:
# print cluster numbers and DNs distribution
iterNum = 4
consensusTemp = consensusResults[iterNum][1]
labels_unique, counts = np.unique(consensusTemp, return_counts=True)
print(labels_unique, counts)

for i in bodyId_indices:
    print(consensusTemp[i], end=', ')
print()

[1 2 3 4 5] [1705 1286 1260  268 1077]


In [94]:
consensusResults

[(10, array([1, 1, 1, ..., 4, 4, 3])),
 (20, array([1, 1, 1, ..., 4, 4, 3])),
 (50, array([1, 1, 1, ..., 5, 5, 4])),
 (100, array([1, 1, 1, ..., 5, 5, 4])),
 (250, array([1, 1, 1, ..., 5, 5, 4]))]

In [115]:
### save results of consensus clustering over all iterations
# np.save('./data/consensusResults.npy', np.array(consensusResults, dtype=object), allow_pickle=True)

In [116]:
# temp = np.load("./data/consensusResults.npy", allow_pickle=True)

In [81]:
# result = modularity.consensus_modularity(adj_mat, repeats=10)

In [82]:
""" # find cluster 0 across all runs to see which neurons are consistently assigned to cluster 0
cluster0_assignments = []
for i in range(10):
    cluster0_assignments.append(np.where(clusterAssignments[i] == 0)[0])

cluster0_assignments = np.array(cluster0_assignments)

# use cluster0 ids to find the corresponding neuron bodyIds
cluster0_bodyIds = []
for i in range(10):
    cluster0_bodyIds.append(graph.names[cluster0_assignments[i]])

# identify which neurons are consistently assigned to cluster 0 by comparing each cluster0_bodyIds to the rest
print(set(cluster0_bodyIds[0]).intersection(*cluster0_bodyIds[1:]))
for i in range(1, 9):
    print(set(cluster0_bodyIds[i]).intersection(*cluster0_bodyIds[:i], *cluster0_bodyIds[i+1:]))
print(set(cluster0_bodyIds[9]).intersection(*cluster0_bodyIds[:9])) """

' # find cluster 0 across all runs to see which neurons are consistently assigned to cluster 0\ncluster0_assignments = []\nfor i in range(10):\n    cluster0_assignments.append(np.where(clusterAssignments[i] == 0)[0])\n\ncluster0_assignments = np.array(cluster0_assignments)\n\n# use cluster0 ids to find the corresponding neuron bodyIds\ncluster0_bodyIds = []\nfor i in range(10):\n    cluster0_bodyIds.append(graph.names[cluster0_assignments[i]])\n\n# identify which neurons are consistently assigned to cluster 0 by comparing each cluster0_bodyIds to the rest\nprint(set(cluster0_bodyIds[0]).intersection(*cluster0_bodyIds[1:]))\nfor i in range(1, 9):\n    print(set(cluster0_bodyIds[i]).intersection(*cluster0_bodyIds[:i], *cluster0_bodyIds[i+1:]))\nprint(set(cluster0_bodyIds[9]).intersection(*cluster0_bodyIds[:9])) '

In [49]:
DIR = "./data/consensusResults2"

In [50]:
# aggregate consensus clustering results from different iterations into 1 numpy array

consensusResults = []
for file in os.listdir(DIR):
    if file.endswith(".npy") and not file.startswith("consensusResults"):
        temp = np.load(f"{DIR}/{file}", allow_pickle=True)
        consensusResults.append(temp[0])

consensusResults = np.array(consensusResults)
# sort rows of 2d numpy array by 1st column
consensusResults = consensusResults[consensusResults[:,0].argsort()]

In [8]:
# save aggregated consensus results in DIR
np.save(f"{DIR}/consensusResults.npy", consensusResults, allow_pickle=True)

In [ ]:
# check if temp and consensusResults have the same elements
""" temp = np.load(f"{DIR}/consensusResults.npy", allow_pickle=True)
for i in range(len(consensusResults)):
    print((consensusResults[i][1] == temp[i][1]).all()) """

In [51]:
# curate all consensus results into a CSV file with columns: Num Iterations, Clustering consistency, DNp cluster assignment
# Clustering consistency is the number of unique cluster assignments and their counts for an iteration

df = pd.DataFrame(columns=['Num Iterations', 'Clustering consistency', 'DNp Cluster Assignment'])

for i in range(len(consensusResults)):
    labels_unique, counts = np.unique(consensusResults[i][1], return_counts=True)
    df.loc[i] = [consensusResults[i][0], counts, consensusResults[i][1][bodyId_indices]]

In [52]:
df

,Num Iterations,Clustering consistency,DNp Cluster Assignment
0,10,"[1970, 1275, 1268, 1083]","[4, 4, 1, 4, 4, 2, 1, 2, 2, 1, 1, 4]"
1,25,"[1700, 1281, 70, 410, 1086, 1049]","[5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5]"
2,50,"[1710, 1286, 180, 263, 1077, 1080]","[5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5]"
3,75,"[1707, 1288, 1261, 263, 1077]","[5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5]"
4,100,"[1705, 1285, 448, 1079, 1079]","[4, 4, 1, 4, 4, 2, 1, 2, 2, 1, 1, 4]"
5,150,"[1710, 1290, 1263, 252, 1081]","[5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5]"
6,200,"[1705, 1293, 1262, 259, 1077]","[5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5]"
7,250,"[1708, 1286, 1259, 268, 1075]","[5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5]"
8,300,"[1708, 1287, 1260, 265, 1076]","[5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5]"
9,350,"[1705, 1286, 444, 1078, 1083]","[4, 4, 1, 4, 4, 2, 1, 2, 2, 1, 1, 4]"


In [53]:
# save df as CSV file
df.to_csv(f"{DIR}/consensusSummary.csv", index=False)

In [8]:
DNp_cluster_assignments = [int(i) for i in "5 5 1 5 5 2 1 2 2 1 1 5".split(" ")]
DNp_cluster_assignments

[5, 5, 1, 5, 5, 2, 1, 2, 2, 1, 1, 5]

In [11]:
# extract bodyIds of neurons in cluster 1,2,5
cluster1_bodyIds, cluster2_bodyIds, cluster5_bodyIds = [], [], []

for i in range(len(DNp_cluster_assignments)):
    if DNp_cluster_assignments[i] == 1:
        cluster1_bodyIds.append(DNp_ids[i])
    elif DNp_cluster_assignments[i] == 2:
        cluster2_bodyIds.append(DNp_ids[i])
    elif DNp_cluster_assignments[i] == 5:
        cluster5_bodyIds.append(DNp_ids[i])

In [12]:
cluster1_bodyIds, cluster2_bodyIds, cluster5_bodyIds

([1565846637, 1100404581, 512851433, 5813026936],
 [5813023322, 1226887763, 1228264951],
 [2307027729, 5813024015, 1405231475, 1466998977, 1281324958])

Cluster 1: 1565846637 (DNp03), 1100404581 (DNp07), 512851433 (DNp10(PDM27)_L), 5813026936 (DNp10_R)
Cluster 2: 5813023322 (DNp06), 1226887763 (DNp09), 1228264951 (DNp09)
Cluster 5: 2307027729 (Giant Fiber), 5813024015 (DNp02), 1405231475 (DNp04), 1466998977 (DNp05), 1281324958 (DNp11)